In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a6c87fb1a9c12811a3c88753b04fe9c6c246abbaa789a5dc16fac760b17ab215
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
import pandas as pd
import random
import os
import sys
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.ml.feature import Bucketizer
import copy
import pandas as pd
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

**促活场景，可以不发优惠券**

In [4]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [60, 60, 40, -1] * 4
is_convert = []
control_convert = []

In [5]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = random.uniform(0, 0.005)
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [6]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [7]:
df_old = pd.DataFrame(data)
df_old['expected_coupon_threshold_uplift'] = (df_old['is_convert'] - df_old['control_convert']) * df_old['coupon_threshold']
df_old['expected_coupon_reduce'] = df_old['coupon_reduce']

In [8]:
df_old

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expected_coupon_threshold_uplift,expected_coupon_reduce
0,3790260,2060114740,20,60,0.059819,0.001810,3.480527,20
1,3790260,1806204031,10,60,0.173746,0.001810,10.316157,10
2,3790260,4096652575,10,40,0.270893,0.001810,10.763316,10
3,3790260,1234832524,0,-1,0.001810,0.001810,-0.000000,0
4,4687825,2060114740,20,60,0.176561,0.000119,10.586559,20
5,4687825,1806204031,10,60,0.214746,0.000119,12.877667,10
6,4687825,4096652575,10,40,0.192932,0.000119,7.712548,10
7,4687825,1234832524,0,-1,0.000119,0.000119,-0.000000,0
8,5077519,2060114740,20,60,0.275431,0.004548,16.252952,20
9,5077519,1806204031,10,60,0.112352,0.004548,6.468229,10


**拉新场景，都需要发优惠券**

In [9]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [98, 98, 68, -1] * 4
is_convert = []
control_convert = []

In [10]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = 0
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [11]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [12]:
df_new = pd.DataFrame(data)
df_new['expected_coupon_threshold_uplift'] = (df_new['is_convert'] - df_new['control_convert']) * df_new['coupon_threshold']
df_new['expected_coupon_reduce'] = df_new['coupon_reduce']

In [13]:
df_new

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expected_coupon_threshold_uplift,expected_coupon_reduce
0,3790260,2060114740,20,98,0.191252,0,18.742682,20
1,3790260,1806204031,10,98,0.207185,0,20.304136,10
2,3790260,4096652575,10,68,0.151492,0,10.301460,10
3,3790260,1234832524,0,-1,0.000000,0,-0.000000,0
4,4687825,2060114740,20,98,0.032649,0,3.199606,20
5,4687825,1806204031,10,98,0.068399,0,6.703063,10
6,4687825,4096652575,10,68,0.271954,0,18.492881,10
7,4687825,1234832524,0,-1,0.000000,0,-0.000000,0
8,5077519,2060114740,20,98,0.233389,0,22.872091,20
9,5077519,1806204031,10,98,0.254644,0,24.955114,10


**转为pyspark计算**

In [14]:
# 初始化spark会话
spark = SparkSession.builder.getOrCreate()

In [15]:
spark_df_old = spark.createDataFrame(df_old)
spark_df_old.show()

+-------+----------+-------------+----------------+--------------------+--------------------+--------------------------------+----------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|     control_convert|expected_coupon_threshold_uplift|expected_coupon_reduce|
+-------+----------+-------------+----------------+--------------------+--------------------+--------------------------------+----------------------+
|3790260|2060114740|           20|              60| 0.05981881875751535|0.001810030408139...|              3.4805273009625237|                    20|
|3790260|1806204031|           10|              60| 0.17374598216910098|0.001810030408139...|               10.31615710565766|                    10|
|3790260|4096652575|           10|              40|  0.2708929289106016|0.001810030408139...|              10.763315940098465|                    10|
|3790260|1234832524|            0|              -1|0.001810030408139...|0.001810030408139...|       

In [16]:
spark_df_new = spark.createDataFrame(df_new)
spark_df_new.show()

+-------+----------+-------------+----------------+--------------------+---------------+--------------------------------+----------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|control_convert|expected_coupon_threshold_uplift|expected_coupon_reduce|
+-------+----------+-------------+----------------+--------------------+---------------+--------------------------------+----------------------+
|3790260|2060114740|           20|              98| 0.19125185786060447|              0|               18.74268207033924|                    20|
|3790260|1806204031|           10|              98| 0.20718506402785533|              0|               20.30413627472982|                    10|
|3790260|4096652575|           10|              68| 0.15149205545544758|              0|              10.301459770970435|                    10|
|3790260|1234832524|            0|              -1|                 0.0|              0|                            -0.0|         

In [19]:
result_df_new = spark_df_new.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('expected_coupon_reduce').alias('expected_coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_coupon_threshold_uplift').alias('expected_coupon_threshold_uplift'),).cache()

result_df_new.show()

+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+
|user_id|          is_convert|           coupon_id|expected_coupon_reduce|coupon_threshold|expected_coupon_threshold_uplift|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+
|3790260|[0.19125185786060...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [18.7426820703392...|
|5077519|[0.23338868590053...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [22.8720912182525...|
|4687825|[0.03264903849631...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [3.19960577263878...|
|7927329|[0.02596714475768...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [2.54478018625307...|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+


In [20]:
result_df_old = spark_df_old.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('expected_coupon_reduce').alias('expected_coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_coupon_threshold_uplift').alias('expected_coupon_threshold_uplift'),).cache()

result_df_old.show()

+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+
|user_id|          is_convert|           coupon_id|expected_coupon_reduce|coupon_threshold|expected_coupon_threshold_uplift|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+
|3790260|[0.05981881875751...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [3.48052730096252...|
|5077519|[0.27543059772819...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [16.2529515436254...|
|4687825|[0.17656122457268...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [10.5865591724219...|
|7927329|[0.17918278480222...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [10.4694895894383...|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+


**MCKP求解**

In [21]:
epoch = 1000
B = 50
C = 150
alpha = 0.01
lr_decay = 0.98

In [22]:
def MCKP(df, epoch, B, C, alpha, lr_decay=1):
    N = df.count()
    lambda_B = 0
    lambda_C = 0

    for _ in tqdm(range(epoch)):

        temp = df.cache()

        # 固定lambda，更新xij，选择收益最大的优惠券
        temp = temp.withColumn('score', F.expr('zip_with(is_convert, expected_coupon_reduce, (x, y) -> (x - %f * y))' % (lambda_B)))
        temp = temp.withColumn('score', F.expr('zip_with(score, expected_coupon_threshold_uplift, (x, y) -> (x + %f * y))' % (lambda_C)))
        temp = temp.withColumn('choice', F.expr('array_position(score, array_max(score)) - 1'))

        # 固定优惠券，更新lambda
        delta_B = temp.select(F.sum(F.col('expected_coupon_reduce')[F.col('choice')])).collect()[0][0]
        lambda_B = max(0, lambda_B - alpha * (B - delta_B)/N)

        delta_C = temp.select(F.sum(F.col('expected_coupon_threshold_uplift')[F.col('choice')])).collect()[0][0]
        lambda_C = max(0, lambda_C - alpha * (delta_C - C)/N)

        if _ % 50 == 0:
            print("epoch: {0}, lambda_B: {1}, lambda_C: {2}".format(_, lambda_B, lambda_C))

        if _ % 10 == 0:
          alpha *= lr_decay

        temp.unpersist()

    return temp

In [23]:
result_old = MCKP(result_df_old, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:07<1:57:59,  7.09s/it]

epoch: 0, lambda_B: 0.025, lambda_C: 0.2490914389932059


  5%|▌         | 51/1000 [03:45<47:27,  3.00s/it]

epoch: 50, lambda_B: 1.2019702391999996, lambda_C: 11.976019860373443


 10%|█         | 101/1000 [06:07<40:34,  2.71s/it]

...

 90%|█████████ | 901/1000 [41:43<03:31,  2.13s/it]

epoch: 900, lambda_B: 10.286695474361391, lambda_C: 102.49311112774382


 95%|█████████▌| 951/1000 [43:46<02:15,  2.77s/it]

epoch: 950, lambda_B: 10.477730188903688, lambda_C: 104.39651560546397


100%|██████████| 1000/1000 [45:50<00:00,  2.75s/it]


In [24]:
result_old = result_old.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [25]:
result_old.show()

+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|expected_coupon_reduce|coupon_threshold|expected_coupon_threshold_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+--------------------+------+-------------+
|3790260|[0.05981881875751...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [3.48052730096252...|[156.297549119424...|     2|   4096652575|
|5077519|[0.27543059772819...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [16.2529515436254...|[1511.04119217400...|     0|   2060114740|
|4687825|[0.17656122457268...|[2060114740, 1806...|       [20, 10, 10, 0]|[60, 60, 40, -1]|            [10.5865591724219...|[910.015892768554...|     1|   1806204031

In [26]:
result_new = MCKP(result_df_new, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:03<53:33,  3.22s/it]

epoch: 0, lambda_B: 0, lambda_C: 0.1480695067903923


  5%|▌         | 51/1000 [02:12<41:42,  2.64s/it]

epoch: 50, lambda_B: 0, lambda_C: 7.119005619802953


 10%|█         | 101/1000 [04:17<34:04,  2.27s/it]

...

 90%|█████████ | 901/1000 [38:40<03:41,  2.23s/it]

epoch: 900, lambda_B: 0, lambda_C: 60.92583701566637


 95%|█████████▌| 951/1000 [40:46<02:13,  2.73s/it]

epoch: 950, lambda_B: 0, lambda_C: 62.05729365415136


100%|██████████| 1000/1000 [42:50<00:00,  2.57s/it]


In [27]:
result_new = result_new.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [28]:
result_new.show()

+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|expected_coupon_reduce|coupon_threshold|expected_coupon_threshold_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+----------------------+----------------+--------------------------------+--------------------+------+-------------+
|3790260|[0.19125185786060...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [18.7426820703392...|[1181.74430520919...|     1|   1806204031|
|5077519|[0.23338868590053...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [22.8720912182525...|[1442.10756197850...|     1|   1806204031|
|4687825|[0.03264903849631...|[2060114740, 1806...|       [20, 10, 10, 0]|[98, 98, 68, -1]|            [3.19960577263878...|[201.738251043272...|     2|   4096652575